<a href="https://colab.research.google.com/github/jackashore/judgement/blob/develop/Ngrams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import import_ipynb
import main
import Preprocessing
from main import process_judgement
from Preprocessing import normalize
import pandas as pd
import os
from nltk.util import everygrams
from collections import Counter

importing Jupyter notebook from main.ipynb
importing Jupyter notebook from preprocessing.ipynb
importing Jupyter notebook from meta_extraction.ipynb
importing Jupyter notebook from names_hider.ipynb
importing Jupyter notebook from Preprocessing.ipynb


In [2]:
"""import os
import re
import nltk
import pandas as pd
import random
from bs4 import BeautifulSoup
from string import punctuation
from pymorphy2 import MorphAnalyzer
from nltk.collocations import *
from nltk.corpus import stopwords
from nltk.util import everygrams

stops = set(stopwords.words('russian'))
punct = punctuation+'«»—…“”*№–).'
morph = MorphAnalyzer()"""

"import os\nimport re\nimport nltk\nimport pandas as pd\nimport random\nfrom bs4 import BeautifulSoup\nfrom string import punctuation\nfrom pymorphy2 import MorphAnalyzer\nfrom nltk.collocations import *\nfrom nltk.corpus import stopwords\nfrom nltk.util import everygrams\n\nstops = set(stopwords.words('russian'))\npunct = punctuation+'«»—…“”*№–).'\nmorph = MorphAnalyzer()"

In [3]:
# nltk.download('stopwords')

In [2]:
court1 = pd.DataFrame()

In [5]:
"""# препроцессинг

meta_data = {'court': '', 'judge': '', 'prosecutor': '', 'secretary': '', 'accused': '', 'result': '', 'category': '', 
            'punishment_type': '', 'punishment_term': ''}

def parse_xml(path):   # парсим xml
    with open(path, 'r', encoding='utf-8')as f:
        soup = BeautifulSoup(f.read(), 'html.parser')
        
        meta_data['court'] = soup.court.string     # добавляем в метаданные то, что имеется в разметке
        meta_data['judge'] = soup.judge.string
        meta_data['result'] = soup.result.string
        meta_data['category'] = soup.category.string
        
        html = []
        for line in soup.body:
            line = line.replace('[', '')
            line = line.replace(']', '') 
            html.append(line)
        true_html = ' '.join(html)
                
        html_soup = BeautifulSoup(true_html, 'html.parser')
        return html_soup.get_text()
    
def get_parts(text):    # делим на части
    lines = [line for line in text.split('\n')]
    beg, end = 0, 0 
    for num, line in enumerate(lines):
        if re.search(r'у\s*с\s*т\s*а\s*н\s*о\s*в\s*и\s*л', line.lower()):
            beg = num + 1
        
        if re.search(r'п\s*р\s*и\s*г\s*о\s*в\s*о\s*р\s*и\s*л', line.lower()) \
        or re.search(r'п\s*о\s*с\s*т\s*а\s*н\s*о\s*в\s*и\s*л', line.lower()):
        #if re.search(r'[приговорил ]{10,}', line.lower()) or re.search(r'[постановил ]{10,}', line.lower()):
        #if 'приговорил' in line.lower() or 'п р и г о в о р и л' in line.lower() or 'постановил'\
         #in line.lower() or 'п о с т а н о в и л' in line.lower():
            end = num + 1

        if beg and end:
            beginning = [stroka.strip() for stroka in lines[:beg]]
            main_part = [stroka.strip() for stroka in lines[beg:end]]
            ending = [stroka.strip() for stroka in lines[end:]]
            return [' '.join(main_part), ' '.join(beginning), ' '.join(ending)]
                
        
def clean(text):
    pattern1 = re.compile(r'(п|ч|ст)(\.|\s|\d)')             # обозначение статьи (пункт,часть,статья)
    pattern2 = re.compile(r'((У|Г)П?К|КоАП|ПДД)\sРФ')     # названия кодексов
    pattern3 = re.compile('\d')                            # цифры
    pattern4 = re.compile('(ДД.ММ.ГГ|дд.мм.гг)')           # даты

    text = re.sub(pattern1, ' ABBR ', text)
    text = re.sub(pattern2, ' DOCUMENT ', text)
    text = re.sub(pattern3, ' DIGIT ', text)
    text = re.sub(pattern4, ' DATE ', text)
    text = text.replace('\xad', '')
    text = text.replace('п р и г о в о р и л', '')
    text = text.replace('п о с т а н о в и л', '')
    text = text.replace('П Р И Г О В О Р И Л', '')
    text = text.replace('П О С Т А Н О В И Л', '')
    return text
        
def process_xml(path):
    try:
        return clean(get_parts(parse_xml(path))[0])
    except TypeError:
        return None"""

"# препроцессинг\n\nmeta_data = {'court': '', 'judge': '', 'prosecutor': '', 'secretary': '', 'accused': '', 'result': '', 'category': '', \n            'punishment_type': '', 'punishment_term': ''}\n\ndef parse_xml(path):   # парсим xml\n    with open(path, 'r', encoding='utf-8')as f:\n        soup = BeautifulSoup(f.read(), 'html.parser')\n        \n        meta_data['court'] = soup.court.string     # добавляем в метаданные то, что имеется в разметке\n        meta_data['judge'] = soup.judge.string\n        meta_data['result'] = soup.result.string\n        meta_data['category'] = soup.category.string\n        \n        html = []\n        for line in soup.body:\n            line = line.replace('[', '')\n            line = line.replace(']', '') \n            html.append(line)\n        true_html = ' '.join(html)\n                \n        html_soup = BeautifulSoup(true_html, 'html.parser')\n        return html_soup.get_text()\n    \ndef get_parts(text):    # делим на части\n    lines = [l

In [3]:
import os

def get_texts(main_path):  # получаем тексты приговоров
    raw_texts = []
    for items in os.walk(main_path):
        for file in items[2]:
            print(file)
            if file.split('.')[~0] == 'xml':
                file_path = os.path.join(items[0], file)
                #raw_texts.append(process_xml(file_path))
                raw_texts.append(process_judgement(file_path))
    return raw_texts

In [9]:
# записываем тексты в датафрейм
# в этом суде больше всего приговоров (6 тысяч)

court1['raw_main_parts'] = get_texts('data/leninskij-rajonnyj-sud-g-barnaula-altajskij-kraj-1102')

.DS_Store
429355694.xml
425417501.xml
429402232.xml
429402226.xml
428933001.xml
428501348.xml
428527600.xml
428542098.xml
428889984.xml
429483248.xml
428490109.xml
428510872.xml
428490096.xml
429333854.xml
428542107.xml
429333840.xml
429332035.xml
429332021.xml
428527589.xml
428527588.xml
428878439.xml
429332020.xml
429332034.xml
429333841.xml
428542112.xml
428542106.xml
429406337.xml
428490097.xml
428807986.xml
428510867.xml
428807992.xml
429483249.xml
428889985.xml
429330541.xml
429330555.xml
428542099.xml
428527601.xml
428933000.xml
428501349.xml
429402227.xml
429402233.xml
425417500.xml
428425214.xml
428899770.xml
428425200.xml
428899772.xml
429361240.xml
425511601.xml
425417502.xml
429402219.xml
428933002.xml
428527603.xml
428889987.xml
429330543.xml
429330557.xml
428889978.xml
429483263.xml
428807984.xml
428510871.xml
428917229.xml
428807990.xml
429397688.xml
428510859.xml
428534938.xml
429333843.xml
428542104.xml
429332022.xml
429332036.xml
425484578.xml
425417489.xml
429392531.

425417491.xml
425417485.xml
428527586.xml
428527592.xml
428527593.xml
425417484.xml
428878436.xml
428534935.xml
428542109.xml
429406338.xml
429397685.xml
428490098.xml
428807989.xml
428917230.xml
428917224.xml
425470033.xml
428889975.xml
428542096.xml
428501346.xml
429402214.xml
428548752.xml
429402228.xml
428864802.xml
428476899.xml
428899769.xml
428864814.xml
428501344.xml
428542094.xml
429330564.xml
428889988.xml
429330558.xml
429483250.xml
428889977.xml
425470031.xml
428917232.xml
428917226.xml
429397687.xml
429397693.xml
429397678.xml
428534937.xml
429333858.xml
425417479.xml
425417486.xml
425417492.xml
428476912.xml
428527591.xml
428527585.xml
428476906.xml
428527590.xml
425417493.xml
425417487.xml
429332038.xml
425417478.xml
428510880.xml
428534936.xml
429397679.xml
428510857.xml
428917227.xml
429338059.xml
428917233.xml
429386600.xml
428889976.xml
429330559.xml
429330565.xml
428542095.xml
428501351.xml
428501345.xml
429402217.xml
428864815.xml
428476898.xml
429355680.xml
105385

425594343.xml
403730164.xml
104668554.xml
429378265.xml
100179487.xml
404188007.xml
428633806.xml
425498545.xml
425576107.xml
404038539.xml
425556599.xml
428579586.xml
106321733.xml
404755559.xml
425523847.xml
105209510.xml
428579579.xml
425663203.xml
428575357.xml
425523853.xml
429355693.xml
404755565.xml
402707813.xml
104028667.xml
404755571.xml
402707807.xml
107019840.xml
425511613.xml
402707811.xml
404755567.xml
107019842.xml
402707805.xml
106825346.xml
425511607.xml
425523845.xml
429355685.xml
429355691.xml
106175333.xml
425523851.xml
428575355.xml
106632237.xml
105209506.xml
425663215.xml
105385528.xml
425706300.xml
425641739.xml
106321731.xml
428579590.xml
425576111.xml
425669937.xml
425576105.xml
104872498.xml
405147944.xml
429349938.xml
403730172.xml
429378273.xml
100179491.xml
404188011.xml
428633804.xml
404188005.xml
429378267.xml
100179485.xml
104668556.xml
404889203.xml
425594341.xml
401912164.xml
405273873.xml
402123097.xml
100179452.xml
403886140.xml
401912158.xml
428630

428626460.xml
106236173.xml
100179552.xml
100179546.xml
428984396.xml
104576175.xml
429242038.xml
428626474.xml
405398050.xml
429353647.xml
429216469.xml
429239105.xml
425626661.xml
404334508.xml
425618943.xml
425626675.xml
429239111.xml
106733997.xml
404478217.xml
404334520.xml
404478203.xml
402326241.xml
428940854.xml
429205603.xml
425611032.xml
425611026.xml
401461629.xml
429198940.xml
425611027.xml
428555147.xml
402520456.xml
401461628.xml
429205602.xml
402520442.xml
425611033.xml
428940855.xml
404478202.xml
404334521.xml
404478216.xml
106733996.xml
404334509.xml
428948574.xml
429239110.xml
429239104.xml
428561209.xml
425626660.xml
428984397.xml
100179547.xml
404618943.xml
429242039.xml
405398045.xml
428626461.xml
425701988.xml
428585400.xml
100179553.xml
429187487.xml
429226583.xml
425649066.xml
106020958.xml
428955928.xml
429187493.xml
429003778.xml
429223362.xml
105969051.xml
425585417.xml
428977586.xml
429216483.xml
425585403.xml
428585399.xml
100179433.xml
403570558.xml
429201

429342343.xml
105049646.xml
429340232.xml
425602889.xml
428990926.xml
428622569.xml
429239097.xml
405755804.xml
429196182.xml
425641740.xml
404038542.xml
428622555.xml
429196196.xml
105546033.xml
425634320.xml
425634334.xml
428808000.xml
105546027.xml
106256271.xml
107126361.xml
105546023.xml
107126359.xml
425634330.xml
425634324.xml
105385541.xml
105546037.xml
107126365.xml
104485369.xml
401218999.xml
107126371.xml
429342353.xml
429239093.xml
405019851.xml
428990922.xml
405147939.xml
425710537.xml
425710523.xml
429340236.xml
405019845.xml
105049642.xml
428970632.xml
106733801.xml
425641750.xml
425641744.xml
428970626.xml
429187504.xml
103473666.xml
429187510.xml
103473672.xml
104730740.xml
104453263.xml
429205597.xml
106283892.xml
428596299.xml
429349951.xml
402889641.xml
428458053.xml
425470045.xml
425470051.xml
403570578.xml
106573047.xml
429003764.xml
425585423.xml
429003770.xml
428984406.xml
104576183.xml
405520162.xml
429201685.xml
428585391.xml
429201691.xml
429223356.xml
428984

...

407188608.xml
425730706.xml
425714698.xml
407188620.xml
401078885.xml
407850435.xml
100179928.xml
425363195.xml
428861812.xml
406615665.xml
400526614.xml
425726819.xml
425780990.xml
100179733.xml
425759029.xml
100179727.xml
425780984.xml
100179726.xml
425759028.xml
425780985.xml
425780991.xml
100179732.xml
400526615.xml
425363194.xml
406615664.xml
429391969.xml
401078884.xml
407188621.xml
425714699.xml
406615670.xml
400812878.xml
100179929.xml
407850434.xml
100179901.xml
425730707.xml
407188609.xml
425730713.xml
100179915.xml
407922505.xml
100180181.xml
407002915.xml
425722840.xml
425722854.xml
100180195.xml
407922511.xml
428475960.xml
407002929.xml
406412428.xml
428475974.xml
100180142.xml
408066419.xml
406514918.xml
425768746.xml
100180156.xml
407278754.xml
408066425.xml
407278740.xml
428876002.xml
425777921.xml
425752237.xml
407096782.xml
425752223.xml
425777935.xml
100180036.xml
103473619.xml
100180022.xml
429259297.xml
103473631.xml
103473625.xml
429259283.xml
100179652.xml
425748

425768725.xml
100180135.xml
100179590.xml
425765567.xml
425792851.xml
407696419.xml
407278737.xml
408207766.xml
408207772.xml
100180109.xml
100180096.xml
100180082.xml
425752240.xml
406908105.xml
100180055.xml
100180041.xml
425786950.xml
406308099.xml
100180069.xml
425748754.xml
408137594.xml
100179631.xml
100179625.xml
429256448.xml
425748740.xml
407775345.xml
425771857.xml
425771843.xml
408345838.xml
100179619.xml
425745258.xml
407994561.xml
406812377.xml
425718822.xml
100179816.xml
100179802.xml
407994549.xml
406812363.xml
100179814.xml
406812375.xml
408413628.xml
100179800.xml
407994563.xml
100179828.xml
425718808.xml
425771855.xml
425771841.xml
100179633.xml
425748756.xml
425748742.xml
100179627.xml
100180057.xml
406908107.xml
425786952.xml
100180043.xml
408608717.xml
100180094.xml
425389699.xml
100180080.xml
407278735.xml
425389706.xml
408207770.xml
100180123.xml
405983276.xml
425768733.xml
425795696.xml
425765571.xml
100179586.xml
425792847.xml
425792853.xml
100179592.xml
100180

425745253.xml
425745247.xml
100179606.xml
100180076.xml
425786967.xml
425786973.xml
406714894.xml
100180062.xml
100180089.xml
100179438.xml
425389690.xml
425795700.xml
425798542.xml
425765578.xml
408207779.xml
100180102.xml
425792866.xml
407696406.xml
407696412.xml
100180116.xml
425804206.xml
408277261.xml
425809722.xml
425783994.xml
408277275.xml
406412440.xml
406908091.xml
407534962.xml
406908085.xml
425765587.xml
100179570.xml
100179799.xml
100179941.xml
406094149.xml
100179955.xml
425774887.xml
425734509.xml
429385948.xml
425774893.xml
100179969.xml
100179982.xml
407366927.xml
425759040.xml
100179996.xml
425755652.xml
100179766.xml
425755646.xml
100179772.xml
100179773.xml
425755647.xml
100179767.xml
425755653.xml
100179997.xml
425759041.xml
407366926.xml
100179983.xml
429385949.xml
425734508.xml
425789937.xml
100179968.xml
425774892.xml
425774886.xml
100179954.xml
406094148.xml
100179940.xml
100179798.xml
425734520.xml
100179571.xml
425765586.xml
406908084.xml
407534963.xml
406908

408066407.xml
406514906.xml
425798534.xml
425752229.xml
407096788.xml
425777917.xml
428489327.xml
428489333.xml
100180000.xml
100180014.xml
429259289.xml
103473607.xml
425762301.xml
425762315.xml
103473613.xml
407096777.xml
100180028.xml
100179664.xml
407775338.xml
407452059.xml
100179670.xml
429256435.xml
100179880.xml
407452071.xml
100179658.xml
429250722.xml
429250736.xml
100179894.xml
429244970.xml
428861789.xml
425738225.xml
100180202.xml
425738231.xml
100180216.xml
100179843.xml
425738219.xml
406202091.xml
406202085.xml
407616648.xml
100179857.xml


NameError: name 'court1' is not defined

In [4]:
%%time
court1['raw_main_parts'] = get_texts('data/40-j-garnizonnyj-voennyj-sud-g-priozersk-territorii-za-predelami-rf-7')

.DS_Store
106185412.xml
106065436.xml
106065437.xml
106065439.xml
106065438.xml
CPU times: user 13.3 s, sys: 50.1 ms, total: 13.3 s
Wall time: 13.4 s


In [5]:
for t in court1['raw_main_parts']:
    print(t)

:
Согласно обвинительному заключению, ФИО DIGIT  обвиняется в том, что он, являясь военнослужащим по призыву, примерно в  DIGIT  часа  DATE ГГ, в спальном помещении роты технического обеспечения войсковой части  DIGIT  DIGIT  DIGIT  DIGIT  DIGIT , дислоцированной в <адрес> Республики Казахстан, будучи недовольным упущениями в служебной деятельности младшего по сроку службы рядового Казакова, разбудил последнего и потребовал встать с кровати с целью «воспитания». На отказ потерпевшего, ФИО DIGIT  желая добиться выполнения своих незаконных требований, действуя из ложно понятых интересов службы, с целью подчинить Казакова своему влиянию и воле и утвердить мнимое превосходство над ним, унизить честь и личное достоинство, в нарушение требований  ABBR  ABBR  DIGIT , DIGIT  DIGIT , DIGIT  DIGIT , DIGIT  DIGIT , DIGIT  DIGIT  DIGIT  УВС РФ и  ABBR  ABBR  DIGIT - DIGIT  ДУ РФ, умышленно нанес Казакову два удара ладонью правой руки в затылочную область головы и два удара кулаком правой руки в об

In [16]:
court1.head()

,raw_main_parts
0,"Согласно обвинительному заключению, ФИО DIGIT ..."
1,"Рядовые Тишин, ФИО DIGIT , ФИО DIGIT и младши..."
2,"DATE ГГ около DIGIT DIGIT часов, в казарме..."
3,"Младший сержант ФИО DIGIT , являясь начальнико..."
4,"Младший сержант Вагабов, являясь начальником п..."


In [60]:
"""def normalize(text):
    words = [word.strip(punct) for word in text.lower().split()] 
    words = [word for word in words if word]
    words = [morph.parse(word)[0].normal_form for word in words if word]

    return words"""

In [17]:
court1 = court1.dropna()

In [20]:
%%time
court1['norm_main_parts'] = court1['raw_main_parts'].apply(normalize)  # нормализуем

CPU times: user 1.19 s, sys: 47.6 ms, total: 1.24 s
Wall time: 1.25 s


In [21]:
court1.head()

,raw_main_parts,norm_main_parts
0,"Согласно обвинительному заключению, ФИО DIGIT ...","[согласно, обвинительный, заключение, фио, dig..."
1,"Рядовые Тишин, ФИО DIGIT , ФИО DIGIT и младши...","[рядовой, тишина, фио, digit, фио, digit, и, м..."
2,"DATE ГГ около DIGIT DIGIT часов, в казарме...","[date, год, около, digit, digit, часы, в, каза..."
3,"Младший сержант ФИО DIGIT , являясь начальнико...","[младший, сержант, фио, digit, являться, начал..."
4,"Младший сержант Вагабов, являясь начальником п...","[младший, сержант, вагабов, являться, начальни..."


In [15]:
def get_ngrams_5000(court):   # выделяем 5000 самых частотных энграмм
    all_texts = []
    colloc_5000 = [] 
    for text in court.norm_main_parts:    # записываем все тексты в одно место
        all_texts.extend(text)
    for col in Counter(list(everygrams(all_texts, min_len=5, max_len=20))).most_common(5000):
        colloc_5000.append(' '.join(col[0]))
    return colloc_5000

In [ ]:
def get_ngrams_2000(court):   # выделяем 2000 самых частотных энграмм
    all_texts = []
    colloc_2000 = [] 
    for text in court.norm_main_parts:    # записываем все тексты в одно место
        all_texts.extend(text)
    for col in Counter(list(everygrams(all_texts, min_len=5, max_len=20))).most_common(2000):
        colloc_2000.append(' '.join(col[0]))
    return colloc_2000

In [16]:
gold_std = get_ngrams(court1)    # 5000 частотных энграмм большого суда

In [18]:
print(*(gold_std[:20]), sep='\n')

в судебный заседание
кодекс российский федерация
в соответствие с
без цель сбыт
степень общественный опасность
сбыт наркотический средство
в связь с
характер и степень
и степень общественный
и степень общественный опасность
хищение чужое имущество
характер и степень общественный
характер и степень общественный опасность
явка с повинный
уголовный кодекс российский
уголовный кодекс российский федерация
по адрес адрес
цель сбыт наркотический
цель сбыт наркотический средство
без цель сбыт наркотический


In [19]:
random.shuffle(os.listdir('data'))     # взяли 10 рандомных судов
ten_courts = os.listdir('data')[:10]

Дальше нужно снова выделить для этих 10 судов их частотные энграммы и по очереди пересечь с `gold_std`

In [20]:
ten_courts

['krasnozerskij-rajonnyj-sud-novosibirskaya-oblast-986',
 'kondinskij-rajonnyj-sud-xanty-mansijskij-avtonomnyj-okrug-927',
 'kulundinskij-rajonnyj-sud-altajskij-kraj-1040',
 'primorskij-rajonnyj-sud-arxangelskaya-oblast-1632',
 'korochanskij-rajonnyj-sud-belgorodskaya-oblast-941',
 'verxnepyshminskij-gorodskoj-sud-sverdlovskaya-oblast-400',
 'saratovskij-oblastnoj-sud-saratovskaya-oblast-2403',
 'ketchenerovskij-rajonnyj-sud-respublika-kalmykiya-834',
 'bogdanovichskij-gorodskoj-sud-sverdlovskaya-oblast-310',
 'severouralskij-gorodskoj-sud-sverdlovskaya-oblast-1768']

In [21]:
court_dict = {court:'court_'+str(i) for i, court in enumerate(ten_courts, 2)}
court_dict

{'krasnozerskij-rajonnyj-sud-novosibirskaya-oblast-986': 'court_2',
 'kondinskij-rajonnyj-sud-xanty-mansijskij-avtonomnyj-okrug-927': 'court_3',
 'kulundinskij-rajonnyj-sud-altajskij-kraj-1040': 'court_4',
 'primorskij-rajonnyj-sud-arxangelskaya-oblast-1632': 'court_5',
 'korochanskij-rajonnyj-sud-belgorodskaya-oblast-941': 'court_6',
 'verxnepyshminskij-gorodskoj-sud-sverdlovskaya-oblast-400': 'court_7',
 'saratovskij-oblastnoj-sud-saratovskaya-oblast-2403': 'court_8',
 'ketchenerovskij-rajonnyj-sud-respublika-kalmykiya-834': 'court_9',
 'bogdanovichskij-gorodskoj-sud-sverdlovskaya-oblast-310': 'court_10',
 'severouralskij-gorodskoj-sud-sverdlovskaya-oblast-1768': 'court_11'}

In [22]:
def get_all_done(dictionary):
    d_ngrams = {}
    for court, df in dictionary.items():
        print(court)
        df = pd.DataFrame()
        df['raw_main_parts'] = get_texts(os.path.join('data', court))
        df = df.dropna()
        print('got texts for %s' % court)
        print(df.shape)
        df['norm_main_parts'] = df['raw_main_parts'].apply(normalize)
        print('%s normalized' % court)
        d_ngrams[court] = get_ngrams(df)
        print('got ngrams for %s' % court)
    return d_ngrams

In [23]:
courts_ngrams = get_all_done(court_dict)

krasnozerskij-rajonnyj-sud-novosibirskaya-oblast-986
got texts for krasnozerskij-rajonnyj-sud-novosibirskaya-oblast-986
(274, 1)
krasnozerskij-rajonnyj-sud-novosibirskaya-oblast-986 normalized
got ngrams for krasnozerskij-rajonnyj-sud-novosibirskaya-oblast-986
kondinskij-rajonnyj-sud-xanty-mansijskij-avtonomnyj-okrug-927
got texts for kondinskij-rajonnyj-sud-xanty-mansijskij-avtonomnyj-okrug-927
(364, 1)
kondinskij-rajonnyj-sud-xanty-mansijskij-avtonomnyj-okrug-927 normalized
got ngrams for kondinskij-rajonnyj-sud-xanty-mansijskij-avtonomnyj-okrug-927
kulundinskij-rajonnyj-sud-altajskij-kraj-1040
got texts for kulundinskij-rajonnyj-sud-altajskij-kraj-1040
(349, 1)
kulundinskij-rajonnyj-sud-altajskij-kraj-1040 normalized
got ngrams for kulundinskij-rajonnyj-sud-altajskij-kraj-1040
primorskij-rajonnyj-sud-arxangelskaya-oblast-1632
got texts for primorskij-rajonnyj-sud-arxangelskaya-oblast-1632
(445, 1)
primorskij-rajonnyj-sud-arxangelskaya-oblast-1632 normalized
got ngrams for primorskij

In [24]:
for ngrams in courts_ngrams.values():
    final_ngrams = []
    final_ngrams.extend(set(gold_std)&set(ngrams))
final_ngrams=set(final_ngrams)
print(final_ngrams)

{'по уголовный кодекс', 'уголовный кодекс российский федерация в', 'не находить основание для', 'не находить основание', 'психотропный вещество и', 'в тот число', 'ходатайство о постановление приговор без проведение судебный', 'и личность виновный', 'есть открытый хищение чужое имущество', 'следующий обстоятельство год около', 'цель сбыт наркотический средство в особо', 'применение насилие не', 'магазин дать изъять', 'год в период', 'крупный размер при', 'средство в особо крупный', 'есть тайный хищение чужое имущество совершенный', 'наказание суд учитывать', 'обстоятельство год около', 'влияние назначить наказание на исправление осудить и на условие жизнь', 'квартира по адрес адрес', 'грабёж то есть открытый', 'на исправление осудить и на условие жизнь', 'осознать характер и', 'россия по год', 'в установленный закон', 'по уголовный дело', 'в состояние алкогольный опьянение', 'о постановление приговор', 'без цель сбыт наркотический средство в особо', 'с часы до', 'наказание суд учитыват

In [25]:
len(final_ngrams)

495

In [26]:
with open('juridisms.txt', 'w', encoding='utf-8') as f:
    for item in final_ngrams:
        f.write(item+'\n')